# **Benign vs. Malignant Skin Lesion Classifier**

# Setting Up Environment

In [ ]:
# Import and configure warnings to ignore minor warnings
from warnings import filterwarnings

# Visualization libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Import numerical operations and array handling library
import numpy as np

# Import core PyTorch libraries for neural network construction
import torch
import torch.nn as nn  # Neural network layers and functions
import torch.optim as optim # Optimization algorithms

# Import sklearn's confusion matrix for model evaluation
from sklearn.metrics import confusion_matrix

# Import modules for analysing results
from sklearn.metrics import classification_report, accuracy_score

# Import torchvision for image-based model utilities
from torchvision import models, datasets, transforms # Pre-built models, data handling, and image transformations

# Data loader utilities for batching, shuffling, and splitting data
from torch.utils.data import DataLoader, random_split

# Utilities for mixed precision training to enhance performance
from torch.cuda.amp import autocast, GradScaler

# Learning rate scheduler to improve training efficiency
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Import metric calculation tools specifically for classification
from torchmetrics.classification import BinaryAccuracy

import copy
filterwarnings('ignore') # Ignore warnings that do not signal breakdown

In [ ]:
# Sets the device for computation to 'cuda' (GPU) if available, otherwise goes back to 'cpu' (Central Processing Unit). 
# This enables optimal use of hardware by leveraging GPU acceleration when possible.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# Cleaning Up Data

In [ ]:
# Define image target size after the following transformations - referred to in the resizing line
ImageDimensions = 170

# Training transformer - These random transformations help in augmenting the 
# data by providing varied orientations and perspectives of the same image, 
# thus helping the model generalise better during training by learning to 
# recognise patterns under various transformations - especially useful for DermaTrack
transformer = transforms.Compose([
    transforms.Resize(size = (ImageDimensions, ImageDimensions), antialias = True), # Resises to ImageDimensions dimensions with antialias to keep image quality high
    transforms.CenterCrop(size = (ImageDimensions, ImageDimensions)),#Crops from the centre of the image to keep the lesion in the image
    # Rotate and Flip for better training
    transforms.RandomRotation(degrees = 20),# Random rotation +/-20 degrees
    transforms.RandomHorizontalFlip(p = 0.3), # Random horizontal flip occurs 30% of the time
    transforms.RandomVerticalFlip(p = 0.3), # Random vertical flip occurs 30% of the time
    # Convert for model 
    transforms.ToTensor(), #Converts all images to tensors, which is a suitable input for the model
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]) # Normalises the tensors using standard mean and std
])

# Test/Validation transformer - does the same as above without the random flips/rotations
# Randomness is exluded to ensure consistency in evaluation so images are simply formatted
valTransformer = transforms.Compose([
    transforms.Resize(size = (ImageDimensions, ImageDimensions), antialias = True),
    transforms.CenterCrop(size = (ImageDimensions, ImageDimensions)),
    
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [ ]:
DATA_PATH = '/kaggle/input'
import os
def get_data_distribution(data):
    data_path = data
    class_counts = {}

    for class_label in os.listdir(data_path):
        class_path = os.path.join(data_path, class_label)
        number_of_images = len(os.listdir(class_path))
        class_counts[class_label] = number_of_images

    for class_name, counts in class_counts.items():
        print(f"Class: {class_name}, Number of images: {counts}")
        
    return class_counts
        
class_counts = get_data_distribution(os.path.join(DATA_PATH, 'train'))

In [ ]:
def plot_distribution(class_counts):
    plt.bar(class_counts.keys(), class_counts.values())
    plt.xlabel('Classes')
    plt.ylabel('Number of images')
    plt.title('Class distribution')
    plt.tight_layout()
    plt.show()
    
plot_distribution(class_counts)

In [ ]:
def visualize_classes(data):
    dict_for_five_item_in_each_class = {}
    for class_label in os.listdir(data):
        class_path = os.path.join(data, class_label)
        items_paths = []
        for idx, item in enumerate(os.listdir(class_path)):
            if idx < 5:
                items_paths.append(os.path.join(class_path, item))
        dict_for_five_item_in_each_class[class_label] = items_paths
    
    return dict_for_five_item_in_each_class

dict_for_five_item_in_each_class = visualize_classes(os.path.join(DATA_PATH, 'train'))

In [ ]:
import PIL

def plot_5_images_from_each_class(dict_for_five_item_in_each_class):
    for class_labels, image_paths in dict_for_five_item_in_each_class.items():
        plt.figure(figsize=(15, 3))
        for i, image_path in enumerate(image_paths):
            img = PIL.Image.open(image_path)
            plt.subplot(1, len(image_paths), i + 1)
            plt.imshow(img)
            plt.title(class_labels)
            plt.axis('off')
    plt.tight_layout()
    plt.show()


plot_5_images_from_each_class(dict_for_five_item_in_each_class)

# Splitting and Loading DataSets

In [ ]:
# Define the path to the training images directory
trainpath = '/kaggle/input/train'#stored locally on kaggle

# Define the path to the testing images directory
testPath = '/kaggle/input/test'#stored locally on kaggle

# Load the traing datasets from above path and apply the transformations 
# from above block
trainData = datasets.ImageFolder(root = trainpath, transform = transformer)
testData = datasets.ImageFolder(root = testPath, transform = valTransformer)

In [ ]:
len(trainData) # Retrieves the total number of images in the trainData dataset, should be in the thousands

In [ ]:
# Define the sizes of each split
total_samples = len(trainData) # Obtains total number of samples
train_size = int(0.9 * total_samples) # Sets training set to 90% of total
val_size = total_samples - train_size # Sets validation set to 10% of total

# Split the dataset randomly into specified sizes
trainData, valData = torch.utils.data.random_split(trainData, [train_size, val_size])

In [ ]:
# Data loaders
batchSize = 32 #happy medium verified through testing

# Configure DataLoader for the training dataset
# Shuffling helps prevent the model from learning the order of data, enhancing generalisation
trainLoader = DataLoader(trainData, batch_size = batchSize, shuffle = True, num_workers = 4)

# Configure DataLoaders for validation and testing datasets
# No shuffling is needed as these loaders are used only for evaluating the model performance consistently
valLoader = DataLoader(valData, batch_size = batchSize, shuffle = False, num_workers = 4)
testLoader = DataLoader(testData, batch_size = batchSize, shuffle = False, num_workers = 4)

In [ ]:
# Visualise a dataloader
train_data, train_label = next(iter(trainLoader)) # Fetches the first batch of data

# Output the shapes of the data and label tensors to verify their dimensions
# This helps in understanding the batch size and the data structure being passed into the model
train_data.shape, train_label.shape

# Setting Up Model

In [ ]:
# Using a pre-trained EfficientNetV2L model with default weights
model = models.efficientnet_b7(weights = 'DEFAULT')

# Modify the final classifier layer for binary classification
# Replace the second layer in the classifier module with a new linear layer that outputs 1 feature
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)

# Display the modified classifier to verify changes
print("Modified classifier:", model.classifier)

In [ ]:
# Define the loss function for binary classification. BCEWithLogitsLoss combines a Sigmoid layer 
# and the BCELoss in one single class, which makes it more numerically stable than using a plain Sigmoid 
# followed by a BCELoss as separate operations.
criterion = nn.BCEWithLogitsLoss()

# Initialise the optimiser with the model parameters and a learning rate of 0.001.
# Adam optimizer is used here because it combines the advantages of two other extensions of stochastic gradient descent.
# Specifically:
# 1. Adaptive Gradient Algorithm (AdaGrad) that maintains a per-parameter learning rate that improves performance 
#    on problems with sparse gradients (e.g., natural language and computer vision problems).
# 2. Root Mean Square Propagation (RMSProp) that also maintains per-parameter learning rates that are adapted 
#    based on the average of recent magnitudes of the gradients for the weight (which means it does well on online 
#    and non-stationary problems).
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0,
                 restore_best_weights=True, path="best_model.pth"):
        # Initialise the early stopping mechanism
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""
        self.path = path # Sets path to save the best model
        self.early_stop = False

    def save_checkpoint(self, model):
        #Save the model's state dictionary
        torch.save(model.state_dict(), self.path)

    def __call__(self, model, val_loss):
        # Chcek if early stopping should be triggered
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                    self.save_checkpoint(model)
                self.early_stop = True
                return True
        return False

In [ ]:
# Initialise the EarlyStopping mechanism to monitor validation loss improvements.
# Sets 'patience' to 10 to allow 10 epochs without improvement before stopping.
# Sets 'min_delta' to 0.0 to consider any decrease in loss as improvement.
early_stopping = EarlyStopping(patience = 10, min_delta = 0.0)

# Training Step

In [ ]:
# Import tqdm for progress bar visualisation
# The 'autonotebook' submodule automatically chooses the appropriate interface (text or graphical).
from tqdm.autonotebook import tqdm

In [ ]:
def train_step(model:torch.nn.Module, 
               dataloader:torch.utils.data.DataLoader, 
               loss_fn:torch.nn.Module, 
               optimizer:torch.optim.Optimizer):
    # Set the model to training mode
    model.train()
    
    # Initilise cumulative loss and accuracy metrics
    train_loss = 0.
    accuracy_train = BinaryAccuracy(threshold = 0.5).to(device)
    
    # Process each batch using the dataloader
    for batch,(X,y) in enumerate(tqdm(dataloader)):
        # Transfers input and labels to the appropriate device
        X,y = X.to(device,dtype=torch.float32), y.to(device,dtype=torch.float32)
        
        # Forward pass through the model while removing extraneous dimensions from the output
        y_pred = model(X).squeeze()
        # Computes the loss between model predictions and true labels
        loss = loss_fn(y_pred, y)
        # Accumulates the loss values for later averaging
        train_loss += loss.detach().cpu().item()
        
        # Clears previous gradients before backwards pass
        optimizer.zero_grad()
        # Computes gradients of the loss with respect to model parameters
        loss.backward()
        # Adjusts model weights based on computed gradients
        optimizer.step()
        
        # Calculates probabilities from logits for accuracy measurements
        y_proba = torch.sigmoid(y_pred)
        # Updates accuracy metric with current batch's results
        accuracy_train.update(y_proba, y)
        
    #Calculates and returns the average loss and accuracy for the epoch    
    train_accuracy = accuracy_train.compute()
    train_loss = train_loss/len(dataloader) # Average loss across all batches
    
    return train_loss, train_accuracy

# Validation Step

In [ ]:
def val_step(model:torch.nn.Module, 
              dataloader:torch.utils.data.DataLoader, 
              loss_fn:torch.nn.Module):
    # Sets the model to evaluation mode
    model.eval()
    
    # Initialises the valdation loss accumulator
    val_loss = 0.
    # Initialises the accuracy metric for validation data, setting the threshold for binary classification
    accuracy_val = BinaryAccuracy(threshold = 0.5).to(device)
    
    # Iterates ove the validation data loader
    for batch,(X,y) in enumerate(tqdm(dataloader)):
        # Transfers input and labels to the appropriate device
        X,y = X.to(device,dtype=torch.float32), y.to(device,dtype=torch.float32)
        # Forward pass; compute the model's predictions for the batch
        y_pred = model(X).squeeze() # Squeeze removes dimensions of size 1 from the tensor
        #Computes the loss between the predictions and actual labels
        loss = loss_fn(y_pred, y)
        # Accumulates the loss over all batches
        val_loss += loss.detach().cpu().item()
        
        # Calculates the probabilities from the model's logits
        y_proba = torch.sigmoid(y_pred)
        #Updates the accuracy metric for the batch
        accuracy_val.update(y_proba, y)
    # Finalises the accuracy computation after processing all batches    
    val_accuracy = accuracy_val.compute()
    val_loss = val_loss/len(dataloader) # Average over all batches
    
    return val_loss, val_accuracy

# Training Loop

In [ ]:
def train(model:torch.nn.Module, 
          train_dataloader:torch.utils.data.DataLoader, 
          val_dataloader:torch.utils.data.DataLoader, 
          loss_fn:torch.nn.Module, 
          optimzier:torch.optim.Optimizer, 
          epochs:int):
        # Initialises the lowest recorded validation loss to infinity for tracking the best model
    best_test_loss = float('inf')
    
    # Dictionary to store metrics across epochs
    results = {'train_loss':[], 
               'train_accuracy':[], 
               'val_loss':[], 
               'val_accuracy':[]}
    
    # Loops through each epoch, monitoring training and validation progress with a progress bar
    for epoch in tqdm(range(epochs)):
        # Performs a training step and captures trainign loss and accuracy
        train_loss, train_accuracy = train_step(model = model, 
                                                dataloader = train_dataloader, 
                                                loss_fn = loss_fn, 
                                                optimizer = optimizer)
        
        # Performs a validation step and captures validation loss and accuracy
        val_loss, val_accuracy = val_step(model = model, 
                                             dataloader = val_dataloader, 
                                             loss_fn = loss_fn)
        
        
        # Prints out epoch metrics to monitor progress
        print(f"Epoch: {epoch+1} | ", 
              f"Train Loss: {train_loss:.4f} | ", 
              f"Train Accuracy: {train_accuracy:.4f} | ", 
              f"val Loss: {val_loss:.4f} | ", 
              f"val Accuracy: {val_accuracy:.4f}")
        
        # Checks if early stopping conditions are met
        early_stopping(model,val_loss)
    
        if early_stopping.early_stop == True:
            print("Early Stopping!!")
            break # If they are it stops
        
        # Saves metrics to the results dictionary for later
        results["train_loss"].append(train_loss)
        results["train_accuracy"].append(train_accuracy.detach().cpu().item())
        results["val_loss"].append(val_loss)
        results["val_accuracy"].append(val_accuracy.detach().cpu().item())
              
    return results   # Returns the collected metrics for all completed epochs

In [ ]:
# Will train for 200 epochs
EPOCHS = 200

# Sets a seed for reproducibility of results
SEED = 42
torch.manual_seed(SEED) # Seed the random number generator for the CPU
torch.cuda.manual_seed(SEED) # Seed the random number generator for CUDA

# Checks if cuda is available
if torch.cuda.is_available():
    model.cuda()
    # Begins training the model using specified parameters
MODEL_RESULTS = train(model, trainLoader, valLoader, criterion, optimizer, EPOCHS)

# Assessing Results

In [ ]:
model.eval() # Switches the model to evaluation mode

# initialises lists to store labels and predictions
allLabels = []
allPreds = []

# Disables gradient calculations for performance
with torch.no_grad():
    # Iterates over batches of data in the test dataset
    for inputs, labels in testLoader:
        # Moves the inputs and labels to the configured device
        inputs, labels = inputs.to(device), labels.to(device)
        # Ensures labels are the corerct shape and type for calculation
        labels = labels.unsqueeze(1).float()

        # Generates model outputs for the current batch
        outputs = model(inputs)
        # Applies a sigmoid activation to convert logits to probabilities andthreshold at 0.5 to obtain binary predictions
        predictions = (torch.sigmoid(outputs) > 0.5).float()

        # Stores and labels predictions, converting tensors to numpy arrays
        allLabels.extend(labels.cpu().numpy())
        allPreds.extend(predictions.cpu().numpy())

# Converts lists of labels and predicitions into numpy arrays for skylearn        
allLabels = np.array(allLabels)
allPreds = np.array(allPreds)

# Prints classification metrics to evaluate the model
print('Classification report', classification_report(allLabels,allPreds))
print('Accuracy', accuracy_score(allLabels,allPreds))

# Calculates and normalises a confusion matrix
matrix = confusion_matrix(allLabels, allPreds)
cm_normalized = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

# Plots the normalised matrix and formats it using seabron
sns.heatmap(cm_normalized, annot = True, fmt = '.2%', cmap = 'Greens', xticklabels = testData.classes, yticklabels = testData.classes, cbar = False)
plt.title('Confusion Matrix - Test Set', fontsize = 16)
plt.xlabel('Predicted Label', fontsize = 14)
plt.ylabel('True Label', fontsize = 14)
plt.show()

In [ ]:
# Extract validation loss and accuracy from MODEL_RESULTS
val_loss = MODEL_RESULTS['val_loss']
val_accuracy = MODEL_RESULTS['val_accuracy']
epochs = range(1, len(val_loss) + 1)

# Plot validation loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.plot(epochs, val_loss, 'bo-', label='Validation Loss')
plt.title('Validation Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot validation accuracy
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.plot(epochs, val_accuracy, 'go-', label='Validation Accuracy')
plt.title('Validation Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Prediction Example

In [ ]:
model1 = models.efficientnet_b7(weights = 'DEFAULT')
model1.classifier[1] = nn.Linear(model1.classifier[1].in_features, 1)
model1.load_state_dict(torch.load('/kaggle/working/best_model.pth',map_location=torch.device('cpu'))) 

from torchvision import transforms
from PIL import Image

image_path = "/kaggle/input/test/Benign/6299.jpg"

In [ ]:
inputs.shape

In [ ]:
# Validation transformer
valTransformer = transforms.Compose([
    transforms.Resize(size = (ImageDimensions, ImageDimensions), antialias = True),
    transforms.CenterCrop(size = (ImageDimensions, ImageDimensions)),
    
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [ ]:
image = Image.open(image_path).convert("RGB")
image = valTransformer(image)

# Reshape the image to add batch dimension
image = image.unsqueeze(0) 

In [ ]:
image.shape

In [ ]:
testData.class_to_idx

In [ ]:
target = ['Benign','Malignant']
with torch.no_grad():
    model1.eval()
    inputs = image.to('cpu')
    outputs = model1(inputs)
    predictions = (torch.sigmoid(outputs) > 0.5).float()
    print(target[int(predictions.item())])